# Decorators

See PEPs [318](https://www.python.org/dev/peps/pep-0318/) and [3129](https://www.python.org/dev/peps/pep-3129/)

In [ ]:
def add(a, b):
    print('Adding a and b')
    return a + b

add(2, 3)

In [ ]:
def add(a, b):
    print('Add')
    print(a + b)

def sub(a, b):
    print('Subtract')
    print(a - b)

def mul(a, b):
    print('Multiply')
    print(a * b)

def div(a, b):
    print('Divide')
    print(a / b)

**DRY!**

In [ ]:
# We already know that a function can be passed, defined in another function or returned...
def debug(f):
    def wrap(a, b):
        print(f.__qualname__)
        f(a, b)
    return wrap

In [ ]:
wrappedadd = debug(add)
wrappedadd(2, 3)

In [ ]:
@debug
def add(a, b):
    print(a + b)

@debug
def sub(a, b):
    print(a - b)

@debug
def mul(a, b):
    print(a * b)
    
@debug
def div(a, b):
    print(a / b)
    
add(2, 3)    

#### Decorators with args

In [ ]:
from functools import wraps
def debug(msg):
    def wrap_decorator(func):
        @wraps(func)
        def wrap_func(*args, **kwargs):
            print('{} {}'.format(msg, func.__qualname__))
            func(*args, **kwargs)
        return wrap_func
    return wrap_decorator

@debug('I AM HERE:')
def add(a, b):
    print(a + b)
    
add(1, 2)

In [ ]:
def debug(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(func.__qualname__)
        print('Im now turning bad: I wont even execute ' + func.__qualname__)
    return wrapper

def debugcls(cls):
    for key, val in vars(cls).items():
        if callable(val):
            setattr(cls, key, debug(val))
    return cls

@debugcls
class Foo:
    def add(self, a, b):
        print(a + b)
    def sub(self, a, b):
        print(a - b)

f = Foo()
f.add(1, 2)

# Common Python Decorator Patterns

- Wrapper layer for simple timing

In [ ]:
import time
from functools import wraps

def awesometimer(f):
    '''Decorator that times a function f'''
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__ + ':', end - start)
        return result
    return wrapper

@awesometimer
def foo(x):
    '''Some computationally intensive function'''
    while x % 10000000 != 0:
        x += 1

foo(1)

- You want to write a logger which, let's assume involves passing args to a decorator

In [ ]:
import logging
from functools import wraps

def logged(level, name=None, msg=None):
    '''Adds logging'''
    def decorate(f):
        logname = name if name else f.__module__
        log = logging.getLogger(logname)
        logmsg = msg if msg else f.__name__
        
        @wraps(f)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return f(*args, **kwargs)
        return wrapper
    return decorate

@logged(logging.CRITICAL, name='ohgodno', msg='Oh God! Nooo!')
def ohgodno():
    print('Executing ohgodno')
    
ohgodno()

# Class as a decorator

(... not decorating a class, that comes later too!)

In [ ]:
class ComplexProcessor(object):
    """Amazing decorator"""
    def __init__(self, f):
        self.f, self.complex_flag = f, 0
        wraps(f)(self)
    
    def complex_method(self):
        """complex processing"""
        self.complex_flag = 5

    def __call__(self, *args, **kwargs):
        """magic happens here"""
        print('State: ', self.complex_flag)
        print('Begin complex processing...')
        self.complex_method()
        print('End complex processing')
        print('State: ', self.complex_flag)
        response = self.f(*args, **kwargs)
        print('Post execution cleanup')
        return response


@ComplexProcessor
def foo4():
    """Some repeated function pattern"""
    print('foo4')

foo4()

# Decorating a class
[PEP 3129](https://www.python.org/dev/peps/pep-3129/)

In [ ]:
def ensure_participation(attr, minimum=1000):
    def decorator(cls):
        name = "__" + attr
        def getter(self):
            return getattr(self, name)
        def setter(self, value):
            if value < minimum:
                raise ValueError(attr + " too less! :-(")
            setattr(self, name, value)
        setattr(cls, attr, GenericDescriptor(getter, setter))
        return cls
    return decorator

class GenericDescriptor(object):
    def __init__(self, getter, setter):
        self.getter = getter
        self.setter = setter

    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        return self.getter(instance)

    def __set__(self, instance, value):
        return self.setter(instance, value)

@ensure_participation('attendees', minimum=1000)
class Pycon(object):
    """Pycon methods"""
    def __init__(self, attendees):
        self.attendees = attendees

pycon = Pycon(10) # will raise ValueError :-)
pycon = Pycon(1000) # will work fine! :-)

In [ ]:
pycon = Pycon(1000)